# Pré-processamento de Dados de Atendimentos de Alunos

Este notebook detalha o processo completo de pré-processamento aplicado ao conjunto de dados de atendimentos de alunos. O objetivo é realizar a limpeza, o tratamento de valores ausentes e a transformação dos dados, preparando-os para análises futuras e para a modelagem de previsão de churn.

**Responsáveis pelo processo:**
* André
* Arthur
* Ju

---
## Passo 1: Importação e Análise Exploratória Inicial

Nesta etapa, damos o primeiro passo fundamental: carregar os dados brutos e realizar uma análise exploratória inicial. Utilizamos a biblioteca `pandas` para importar o arquivo `atendimentos_de_alunos.csv`.

As ações realizadas são:
1.  **Carregamento dos Dados**: Leitura do arquivo CSV para um DataFrame do pandas.
2.  **Visualização Inicial**: Exibição das cinco primeiras linhas do DataFrame com `df.head()` para uma rápida compreensão da estrutura e do conteúdo das colunas.


In [184]:
import os
import pandas as pd

# === Define o caminho do Arquivo ===

file_path = '../data/interim/atendimentos_de_alunos.csv'

if not os.path.exists(file_path):
    raise FileNotFoundError(f"Arquivo não encontrado no caminho: {file_path}")

# === Carregar o CSV ===

try:
    df = pd.read_csv(file_path, sep=',', encoding='utf-8')

except pd.errors.ParserError as e:
    raise ValueError(f"Erro de parsing no arquivo CSV. Verifique a estrutura do arquivo: {e}")
except Exception as e:
    raise RuntimeError(f"Ocorreu um erro inesperado durante a leitura do arquivo: {e}")

# === Exibição ===

# Exibe as primeiras cinco linhas do DataFrame carregado para uma verificação visual.
df.head()

,Grupo % Cursado,Grupo_Acesso,MATRICULAID,DATAMATRICULA,ENCERRAMENTO_CONTRATO,NOME CURSO PADRÃO,Situação Contrato,Documentos Pessoais Pendentes,SITUACAO,Status_Cliente,...,Ouvidoria,Problema Técnico,Processos Secretaria,Reclame aqui,Rematrícula,Retenção,Solicitação de documentos,Suporte de Acesso,Suporte Pedagogico,has_contact
0,01 - 0% Cursado,00 - Sem Acesso,759596788,03/01/2024,8/23/2025,TÉCNICO EM PRODUÇÃO DE MATERIAIS BILINGUE EM L...,Encerrado,NaN,CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
1,01 - 0% Cursado,05 - Acima de 180 Dias,796681465,7/22/2024,1/18/2025,TÉCNICO EM SEGURANÇA DO TRABALHO,Encerrado,"Fotos 3 X 4, Certidão de Nascimento ou Casamento",CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
2,01 - 0% Cursado,00 - Sem Acesso,801527893,08/02/2024,08/02/2025,TÉCNICO EM RECURSOS HUMANOS,Encerrado,NaN,CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
3,01 - 0% Cursado,00 - Sem Acesso,801583326,08/02/2024,08/02/2025,TÉCNICO EM RECURSOS HUMANOS,Vigente,NaN,CURSANDO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
4,01 - 0% Cursado,00 - Sem Acesso,801939681,08/05/2024,08/05/2025,TÉCNICO EM RECURSOS HUMANOS,Encerrado,"Comprovante de Endereço, Certidão de Nasciment...",CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM


In [185]:
# === Tipos de Dados das Colunas ===
df.dtypes.to_frame(name='Tipo de Dado')

,Tipo de Dado
Grupo % Cursado,object
Grupo_Acesso,object
MATRICULAID,int64
DATAMATRICULA,object
ENCERRAMENTO_CONTRATO,object
NOME CURSO PADRÃO,object
Situação Contrato,object
Documentos Pessoais Pendentes,object
SITUACAO,object
Status_Cliente,object


---
## Passo 2: Diagnóstico de Valores Ausentes

Antes de tratar os dados, é essencial identificar a extensão dos valores ausentes (nulos ou `NaN`) em nosso conjunto de dados. A célula a seguir calcula a quantidade de valores nulos para cada coluna.

Essa análise nos permite:
* **Identificar colunas problemáticas**: Colunas com uma alta porcentagem de valores ausentes podem precisar de estratégias de preenchimento mais sofisticadas ou até mesmo serem removidas.
* **Planejar a estratégia de limpeza**: A quantidade de nulos orienta a escolha das técnicas de imputação (preenchimento) que serão aplicadas na próxima etapa.

In [186]:
# === Criação da Tabela de Quantidade de Valores Ausentes ===

# Calcula o número de valores nulos (`NaN`) em cada coluna do DataFrame.
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

# Apresenta a contagem de nulos em um formato de DataFrame para facilitar a leitura.
pd.DataFrame(null_counts, columns=['Valores Ausentes'])

,Valores Ausentes
Ouvidoria,17392
Estágio,17389
Processos Secretaria,17387
Suporte Pedagogico,17377
Rematrícula,17376
Correção - Plataforma,17368
Correção cadastral,17336
Diploma,17331
Reclame aqui,17309
Retenção,17299


---
### Estratégia de Imputação de Valores Ausentes

Para garantir a qualidade e a consistência dos dados, desenvolvemos a função `fill_missing_values`. Esta função aplica diferentes estratégias de preenchimento de valores nulos com base na natureza de cada coluna, uma abordagem fundamental para preparar o dataset para a modelagem.

As estratégias adotadas são:

-   **Colunas Categóricas (Tipo `object`)**: Os valores nulos são substituídos pela string **'Não Informado'**. Esta técnica cria uma categoria distinta para dados ausentes, permitindo que os algoritmos de machine learning interpretem a ausência de informação como uma característica relevante.

-   **Colunas Numéricas (Quantitativas)**: Os valores nulos são preenchidos com **0**. Esta decisão se baseia na premissa de que, no contexto deste dataset (ex: contagem de atendimentos), um valor ausente significa a não ocorrência do evento.

-   **Colunas de Data**: O tratamento de datas é mais complexo para evitar distorções:
    1.  **Criação de Flag**: Uma nova coluna booleana (ex: `DATAMATRICULA_is_missing`) é criada para registrar se a data original estava ausente. Isso preserva a informação da ausência, que pode ser um preditor importante.
    2.  **Imputação pela Mediana**: Os valores ausentes na coluna de data são preenchidos com a **mediana** das datas existentes. A mediana é usada por ser menos sensível a valores extremos (outliers) do que a média.

In [187]:
def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preenche valores ausentes (NaN) em colunas específicas de um DataFrame.

    Esta função aplica estratégias de imputação baseadas no tipo de dado inferido
    e no contexto da informação, usando 'Não Informado' para colunas categóricas/objeto
    e 0 para colunas numéricas, para indicar a ausência de ocorrência.
    Para colunas de data, cria uma flag de ausência e preenche com a mediana.
    """

    # ID de Matrícula não deve ser Nulo e apresenta apenas um NaN nos dados. Opcionalmente, remover.
    df['MATRICULAID'] = df['MATRICULAID'].fillna(-1)

    # --- 1. Preenchimento de Colunas Categóricas/Texto ---
    df['Grupo % Cursado'] = df['Grupo % Cursado'].fillna('Não Informado')
    df['Grupo_Acesso'] = df['Grupo_Acesso'].fillna('Não Informado')
    df['ENCERRAMENTO_CONTRATO'] = df['ENCERRAMENTO_CONTRATO'].fillna('Não Informado')
    df['NOME CURSO PADRÃO'] = df['NOME CURSO PADRÃO'].fillna('Não Informado')
    df['Situação Contrato'] = df['Situação Contrato'].fillna('Não Informado')
    df['Documentos Pessoais Pendentes'] = df['Documentos Pessoais Pendentes'].fillna('Não Informado')
    df['SITUACAO'] = df['SITUACAO'].fillna('Não Informado')
    df['Status_Cliente'] = df['Status_Cliente'].fillna('Não Informado')
    df['fezPrimeiroAcesso'] = df['fezPrimeiroAcesso'].fillna('Não Informado')
    df['Forma de Pagamento Oficial'] = df['Forma de Pagamento Oficial'].fillna('Não Informado')
    df['ESTADO'] = df['ESTADO'].fillna('Não Informado')


    # --- 2. Tratamento de Colunas Numéricas com Preenchimento de String ---
    df['PercentualConclusao'] = df['PercentualConclusao'].fillna(0)
    df['% Docs Pessoais'] = df['% Docs Pessoais'].fillna(0)


    # --- 3. Preenchimento de Colunas Numéricas com 0 ---
    df['DisciplinasAprovadas'] = df['DisciplinasAprovadas'].fillna(0)
    df['DisciplinasTotais'] = df['DisciplinasTotais'].fillna(0)
    df['# parcelas Vencidas'] = df['# parcelas Vencidas'].fillna(0)


    # --- 4. Preenchimento de Colunas Relacionadas a Atendimentos e Outras Métricas com 0 ---
    df['Total _Atendimentos'] = df['Total _Atendimentos'].fillna(0)
    df['Acesso ao Portal'] = df['Acesso ao Portal'].fillna(0)
    df['Anexar Documentos'] = df['Anexar Documentos'].fillna(0)
    df['Apoio Pedagogico'] = df['Apoio Pedagogico'].fillna(0)
    df['Bot de Atendimento'] = df['Bot de Atendimento'].fillna(0)
    df['Contato Via Ligação'] = df['Contato Via Ligação'].fillna(0)
    df['Correção - Plataforma'] = df['Correção - Plataforma'].fillna(0)
    df['Correção cadastral'] = df['Correção cadastral'].fillna(0)
    df['Diploma'] = df['Diploma'].fillna(0)
    df['Disparos'] = df['Disparos'].fillna(0)
    df['Duvidas Gerais'] = df['Duvidas Gerais'].fillna(0)
    df['Erro'] = df['Erro'].fillna(0)
    df['Estágio'] = df['Estágio'].fillna(0)
    df['Financeiro'] = df['Financeiro'].fillna(0)
    df['Informações Comercias'] = df['Informações Comercias'].fillna(0)
    df['Onboarding'] = df['Onboarding'].fillna(0)
    df['Outros Atendimentos'] = df['Outros Atendimentos'].fillna(0)
    df['Ouvidoria'] = df['Ouvidoria'].fillna(0)
    df['Problema Técnico'] = df['Problema Técnico'].fillna(0)
    df['Processos Secretaria'] = df['Processos Secretaria'].fillna(0)
    df['Reclame aqui'] = df['Reclame aqui'].fillna(0)
    df['Rematrícula'] = df['Rematrícula'].fillna(0)
    df['Retenção'] = df['Retenção'].fillna(0)
    df['Solicitação de documentos'] = df['Solicitação de documentos'].fillna(0)
    df['Suporte de Acesso'] = df['Suporte de Acesso'].fillna(0)
    df['Suporte Pedagogico'] = df['Suporte Pedagogico'].fillna(0)

    # --- 5. Preenchimento de Colunas de Data com a Mediana + Flag de Ausência ---
    for col in ['DATAMATRICULA', 'Data de nascimento']:
        # Cria a coluna flag indicando se o valor original era NaN
        df[f'{col}_is_missing'] = df[col].isna().astype(int)

        # Convertendo para datetime, ESPECIFICANDO O FORMATO D-M-Y
        df[col] = pd.to_datetime(df[col], errors='coerce', format='%d/%m/%Y')
        
        # Calcula a mediana apenas dos valores não-nulos
        median_date = df[col].median()

        # Preenche os NaNs com a mediana
        df[col] = df[col].fillna(median_date)

        # Retorna o formato para dd/mm/yyyy
        df[col] = df[col].dt.strftime("%d/%m/%Y")

    return df

---
### Execução da Limpeza e Verificação Final

Nesta etapa, aplicamos as operações de limpeza definidas anteriormente:

1.  **Preenchimento de Nulos**: A função `fill_missing_values` é chamada para tratar todos os valores ausentes no DataFrame.
2.  **Remoção de Duplicatas**: `df.drop_duplicates(inplace=True)` é executado para remover quaisquer linhas que sejam cópias exatas umas das outras, garantindo a unicidade dos registros.
3.  **Verificação**: Uma checagem final de valores nulos é realizada para confirmar que o processo de limpeza foi bem-sucedido.
4.  **Salvando o Resultado**: O DataFrame limpo e processado é salvo em um novo arquivo CSV na pasta `../data/processed/`, separando os dados brutos dos dados prontos para análise.

In [ ]:
# === Limpeza Final === 

# 1. Aplica a função para preencher todos os valores nulos no DataFrame.
df_processed = fill_missing_values(df)

# 2. Remove linhas duplicadas para garantir que cada registro seja único.
df_processed.drop_duplicates(inplace=True)

# 3. Realiza uma verificação final para garantir que não há mais valores ausentes.
remaining_nulls = df_processed.isnull().sum()
remaining_nulls = remaining_nulls[remaining_nulls > 0]

# Confirma o sucesso da operação.
if remaining_nulls.empty:
    print("Verificação concluída: Não há mais valores ausentes no DataFrame.")
else:
    print("Atenção: Ainda existem valores ausentes nas seguintes colunas:")
    display(pd.DataFrame(remaining_nulls, columns=['Valores Ausentes Restantes']))

# 4. Salva o DataFrame processado em um novo arquivo para uso futuro.
output_path = '../data/processed/atendimentos_de_alunos.csv'
df_processed.to_csv(output_path, index=False, sep=';', encoding='utf-8')


print(f"\nDataFrame processado e salvo com sucesso em: {output_path}")

Verificação concluída: Não há mais valores ausentes no DataFrame.

DataFrame processado e salvo com sucesso em: ../data/processed/atendimentos_de_alunos.csv


---
## Passo 3: Análise Exploratória de Variáveis Categóricas

Antes de prosseguir com a limpeza, é crucial entender a natureza das nossas variáveis de texto (categóricas). Nesta etapa, vamos iterar sobre todas as colunas do tipo `object` para analisar:

* **Cardinalidade**: O número de valores únicos em cada coluna. Uma cardinalidade muito alta pode ser um problema para alguns modelos.
* **Distribuição de Frequência**: A contagem e a proporção de cada categoria. Isso nos ajuda a identificar valores muito raros ou categorias desbalanceadas.
* **Consistência**: Verificar a existência de possíveis inconsistências, como categorias duplicadas com grafias diferentes (ex: "Cancelado" vs. "cancelado") ou a presença de espaços em branco.

Esta análise guiará nossas decisões nas etapas de limpeza e engenharia de atributos.

In [ ]:
# Seleciona apenas as colunas do tipo 'object', que correspondem a strings/categorias.
colunas_categoricas = df.select_dtypes(include=['object']).columns

# Define um limite para o número de valores únicos a serem exibidos, para evitar poluir a saída.
LIMITE_EXIBICAO_UNICOS = 15

print(f"Analisando {len(colunas_categoricas)} colunas categóricas...\n")

# Itera sobre cada coluna categórica para análise individual.
for coluna in colunas_categoricas:
    print(f"--- Análise da Coluna: '{coluna}' ---")
    
    # Calcula a cardinalidade (número de valores únicos)
    num_unicos = df[coluna].nunique()
    print(f"Número de valores únicos: {num_unicos}")

    # Exibe os valores únicos se não forem excessivos
    if num_unicos > 0 and num_unicos <= LIMITE_EXIBICAO_UNICOS:
        print(f"Valores únicos: {df[coluna].unique().tolist()}")
    elif num_unicos > LIMITE_EXIBICAO_UNICOS:
        print(f"Valores únicos (amostra): {df[coluna].unique().tolist()[:LIMITE_EXIBICAO_UNICOS]}...")
    
    # Exibe a contagem e a proporção dos valores mais comuns
    print("\nDistribuição de Frequência (Top 10):")
    print(df[coluna].value_counts(dropna=False).head(10).to_string())
    print("\n" + "="*50 + "\n

---
## Passo 4: Engenharia de Atributos - Criação da Variável Alvo (Churn)

Com os dados limpos, o próximo passo crucial é a **engenharia de atributos**. Nesta etapa, criamos a nossa **variável alvo (target)**, que é a variável que queremos prever com nosso modelo de machine learning.

O objetivo é criar uma coluna binária chamada `churn`, onde:
-   **`1`** indica que o aluno evadiu (ex: cancelou, foi reprovado).
-   **`0`** indica que o aluno não evadiu (ex: está cursando, formado).

Esta coluna é fundamental, pois servirá como o "gabarito" para treinar um modelo de classificação.

---
### 4.1. Carregando os Dados Processados

Para garantir que estamos trabalhando com a versão mais limpa e atualizada dos dados, vamos carregar o arquivo que foi salvo na etapa anterior.

In [189]:
# === Define df como o arquivo processado ===

# Define o caminho para o arquivo processado.
processed_file_path = '../data/processed/atendimentos_de_alunos.csv'

# Verifica a existência do arquivo processado.
if not os.path.exists(processed_file_path):
    raise FileNotFoundError(f"Arquivo processado não encontrado: {processed_file_path}")

# Tenta carregar o arquivo CSV processado.
try:
    df_model = pd.read_csv(processed_file_path, sep=';', encoding='utf-8')
except Exception as e:
    raise RuntimeError(f"Ocorreu um erro ao carregar o arquivo processado: {e}")

# Exibe as primeiras linhas para confirmar o carregamento.
print("Arquivo processado carregado com sucesso. Visualização:")
df_model.head()

Arquivo processado carregado com sucesso. Visualização:


,Grupo % Cursado,Grupo_Acesso,MATRICULAID,DATAMATRICULA,ENCERRAMENTO_CONTRATO,NOME CURSO PADRÃO,Situação Contrato,Documentos Pessoais Pendentes,SITUACAO,Status_Cliente,...,Processos Secretaria,Reclame aqui,Rematrícula,Retenção,Solicitação de documentos,Suporte de Acesso,Suporte Pedagogico,has_contact,DATAMATRICULA_is_missing,Data de nascimento_is_missing
0,01 - 0% Cursado,00 - Sem Acesso,759596788,03/01/2024,8/23/2025,TÉCNICO EM PRODUÇÃO DE MATERIAIS BILINGUE EM L...,Encerrado,Não Informado,CANCELADO,Quitado,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SIM,0,0
1,01 - 0% Cursado,05 - Acima de 180 Dias,796681465,06/12/2023,1/18/2025,TÉCNICO EM SEGURANÇA DO TRABALHO,Encerrado,"Fotos 3 X 4, Certidão de Nascimento ou Casamento",CANCELADO,Quitado,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SIM,0,0
2,01 - 0% Cursado,00 - Sem Acesso,801527893,08/02/2024,08/02/2025,TÉCNICO EM RECURSOS HUMANOS,Encerrado,Não Informado,CANCELADO,Quitado,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SIM,0,0
3,01 - 0% Cursado,00 - Sem Acesso,801583326,08/02/2024,08/02/2025,TÉCNICO EM RECURSOS HUMANOS,Vigente,Não Informado,CURSANDO,Quitado,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SIM,0,0
4,01 - 0% Cursado,00 - Sem Acesso,801939681,08/05/2024,08/05/2025,TÉCNICO EM RECURSOS HUMANOS,Encerrado,"Comprovante de Endereço, Certidão de Nasciment...",CANCELADO,Quitado,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SIM,0,0


---
### 4.2. Análise da Coluna `SITUACAO` para Definição do Churn

Para criar nossa variável `churn`, precisamos primeiro entender os diferentes status possíveis na coluna `SITUACAO`. Esta análise nos permitirá agrupar os status que representam churn e os que não representam.

In [190]:
# === Analise da coluna 'SITUACAO' ===

print("--- Análise da Coluna 'SITUACAO' ---")

# Exibe os valores únicos para identificar todas as categorias possíveis.
print("\nValores Únicos Encontrados:")
unique_situations = df_model['SITUACAO'].unique()
for i, value in enumerate(unique_situations, 1):
    print(f"  {i}. {value}")

# Exibe a contagem de cada valor para entender a distribuição dos alunos.
print("\nContagem de Ocorrências por Valor:")
print(df_model['SITUACAO'].value_counts().to_string())

print("\n--- Fim da Análise ---")

--- Análise da Coluna 'SITUACAO' ---

Valores Únicos Encontrados:
  1. CANCELADO
  2. CURSANDO
  3. EVADIDO
  4. FORMADO
  5. CONCLUIDO_REPROVADO
  6. CONCLUIDO

Contagem de Ocorrências por Valor:
SITUACAO
EVADIDO                6722
CURSANDO               5945
CANCELADO              2524
FORMADO                1458
CONCLUIDO               313
CONCLUIDO_REPROVADO     133

--- Fim da Análise ---


---
### 4.3. Mapeamento de `SITUACAO` para a Coluna `churn` e Atualização do CSV

Com base na análise anterior, definimos um dicionário de mapeamento para traduzir os status da coluna `SITUACAO` para a nossa variável alvo binária, `churn`.

**Regra de Negócio para Churn:**
-   **Churn (1)**: Alunos com status `CANCELADO`, `EVADIDO` ou `CONCLUIDO_REPROVADO`. Estes são os casos que queremos prever.
-   **Não Churn (0)**: Alunos com status `CURSANDO`, `FORMADO`, `CONCLUIDO` ou `Não Informado`.

Este passo transforma uma coluna categórica em uma variável numérica alvo. **Crucialmente, após criar esta coluna, o DataFrame será salvo, atualizando o arquivo `atendimentos_de_alunos.csv` no diretório de dados processados para incluir este novo e importante atributo.**

In [191]:
# === Finalização do Processamento === 

# --- Mapeamento de Churn ---
# Define o dicionário que mapeia cada status para um valor de churn (1) ou não-churn (0).
mapeamento_churn = {
    'CONCLUIDO_REPROVADO': 1,
    'CANCELADO': 1,
    'EVADIDO': 1,
    'Não Informado': 0,
    'CONCLUIDO': 0,
    'CURSANDO': 0,
    'FORMADO': 0
}

# --- Criação e Validação da Coluna 'churn' ---
df['churn'] = df['SITUACAO'].map(mapeamento_churn)
df.dropna(subset=['churn'], inplace=True)
df['churn'] = df['churn'].astype(int)


# --- Análise da Nova Coluna ---
print("## Análise da Nova Coluna 'churn' ##")
print("\nContagem de Churn (1) vs. Não Churn (0):")
print(df['churn'].value_counts())
print("\nProporção de Churn no Dataset:")
print(df['churn'].value_counts(normalize=True).apply(lambda x: f"{x:.2%}"))
print("-" * 40)


# --- ATUALIZAÇÃO DO ARQUIVO CSV ---
output_file = '../data/processed/atendimentos_de_alunos.csv'

# Salva o DataFrame modificado (agora com a coluna 'churn') de volta ao arquivo original.
try:
    df.to_csv(output_file, index=False, sep=';', encoding='utf-8')
    print(f"\nSUCESSO: O arquivo foi atualizado com a coluna 'churn'.")
    print(f"Local: {output_file}")
except Exception as e:
    print(f"\nERRO: Não foi possível salvar o arquivo. Motivo: {e}")

## Análise da Nova Coluna 'churn' ##

Contagem de Churn (1) vs. Não Churn (0):
churn
1    9379
0    7716
Name: count, dtype: int64

Proporção de Churn no Dataset:
churn
1    54.86%
0    45.14%
Name: proportion, dtype: object
----------------------------------------

SUCESSO: O arquivo foi atualizado com a coluna 'churn'.
Local: ../data/processed/atendimentos_de_alunos.csv


---

#### **Conclusão da Etapa de Mapeamento:**

Finalizamos a etapa de engenharia de atributos criando com sucesso a variável `churn`. O DataFrame agora contém uma coluna alvo clara e binária, essencial para o treinamento de modelos de previsão.

-   **Situações de Churn (1)**: `CANCELADO`, `EVADIDO`, `CONCLUIDO_REPROVADO`.
-   **Situações de Não Churn (0)**: `CURSANDO`, `FORMADO`, `CONCLUIDO`, `Não Informado`.

O código agora tem uma diretriz clara (`0` ou `1`) para os algoritmos de machine learning aprenderem os padrões que levam à evasão de alunos.